# Links

* Good Readings
    + Re-sampling (over/under): https://machinelearningmastery.com/random-oversampling-and-undersampling-for-imbalanced-
    + Good reading about Standardization: https://sebastianraschka.com/Articles/2014_about_feature_scaling.html
    + Standardization vs normalization: https://towardsdatascience.com/normalization-vs-standardization-quantitative-analysis-a91e8a79cebf
    + Scaling: https://scikit-learn.org/stable/modules/preprocessing.html
    + https://machinelearningmastery.com/how-to-configure-the-number-of-layers-and-nodes-in-a-neural-network/
    + How to evaluate the model: https://machinelearningmastery.com/evaluate-skill-deep-learning-models/
    + https://www.microsoft.com/en-us/research/blog/three-mysteries-in-deep-learning-ensemble-knowledge-distillation-and-self-distillation/
    + https://towardsdatascience.com/how-to-handle-large-datasets-in-python-with-pandas-and-dask-34f43a897d55
    + Batch size: https://machinelearningmastery.com/how-to-control-the-speed-and-stability-of-training-neural-networks-with-gradient-descent-batch-size/
    + Model doesn't fit in memory: https://gdmarmerola.github.io/big-data-ml-training/
    + More aboute standardization of test set: https://stats.stackexchange.com/questions/202287/why-standardization-of-the-testing-set-has-to-be-performed-with-the-mean-and-sd
    + How to standardize: https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.StandardScaler.html

# How to run: packages to install

    0- conda install nb_conda
    1- conda install matplotlib 
    2- conda install tensorflow
    3- conda install scikit-learn 
    4- conda install seaborn
    5- pip install imbalanced-learn
        
    Run all initialization cells (View -> Cell Toolbar -> Initialization Cell).
    After that select a dataset to load a learning model.
    Finally train and test it.

# Download and extract dataset

https://drive.google.com/file/d/1yPDkDKCKScE9KiLxZ8rc7Zn8cHWgQIua/view?usp=sharing

# Imports

In [ ]:
import json
import math
import matplotlib
import numpy as np
import pandas as pd
import seaborn
import sklearn
import sklearn.model_selection
import sklearn.preprocessing
import tensorflow as tf
import random
from keras.callbacks import CSVLogger
import os
from enum import Enum
import imblearn
from imblearn.over_sampling import RandomOverSampler
from imblearn.under_sampling import RandomUnderSampler

# Hyperparameters

In [ ]:
class BalanceStrategy(Enum):
    NONE = 0
    WEIGHTS = 1
    OVERSAMPLE = 2
    UNDERSAMPLE = 3

balance_strategy = BalanceStrategy.WEIGHTS #Strategy to handle the imbalanced dataset

# Neural networks are stochastic by design and that the source of randomness can be fixed to make results reproducible.
# Therefore, the most robust way to report results and compare models is to repeat your experiment many times (30+) and use summary statistics.
# Source: https://machinelearningmastery.com/reproducible-results-neural-networks-keras/
# Fix random seed.
#tf.random.set_seed(1234)
#np.random.seed(1234) # Scikit Learn does not have its own global random state but uses the numpy random state instead.

model_name = "Test" #it will create a directory with this name to save the model's weights and training history

batch_size = 32 # is important to ensure that each batch has a decent chance of containing a few positive samples
epochs = 20
learning_rate = 0.001 #Eh?Predictor=0.05, default=0.001
drop_out = 0.05 ##Eh?Predictor=0.05

METRICS = [tf.keras.metrics.TruePositives(name='tp'),
           tf.keras.metrics.FalsePositives(name='fp'),
           tf.keras.metrics.TrueNegatives(name='tn'),
           tf.keras.metrics.FalseNegatives(name='fn'),
           tf.keras.metrics.BinaryAccuracy(name='accuracy'),
           tf.keras.metrics.Precision(name='precision'),
           tf.keras.metrics.Recall(name='recall'),
           tf.keras.metrics.AUC(name='auc')]

# Learning Model

In [ ]:
def make_model(metrics = METRICS, output_bias=None):
    if output_bias is not None:
        output_bias = tf.keras.initializers.Constant(output_bias)
    model = tf.keras.Sequential([tf.keras.layers.Dense(len(selectedFeatures), activation='relu'),
                                 tf.keras.layers.Dropout(drop_out),
                                 tf.keras.layers.Dense(50, activation='relu'),
                                 tf.keras.layers.Dropout(drop_out),
                                 tf.keras.layers.Dense(50, activation='relu'),
                                 tf.keras.layers.Dropout(drop_out),
                                 tf.keras.layers.Dense(1, activation='sigmoid', bias_initializer=output_bias)])
    model.compile(optimizer=tf.keras.optimizers.Adam(lr=learning_rate),
                  loss=tf.keras.losses.BinaryCrossentropy(),
                  metrics=metrics)
    return model

# Selected Features and DRV types

In [ ]:
placementFeatures = ["#Cells", "#CellPins", "#Macros", "#MacroPins", "HorizontalOverflow",
                     "VerticalOverflow",
                     "TileArea", "CellDensity", "MacroDensity", "MacroPinDensity",
                     "Layer1BlkgDensity", "Layer2BlkgDensity", "Layer1PinDensity", "Layer2PinDensity"]

placementNeighborFeatures = ["NeighborTileArea", "NeighborCellArea", "NeighborL1PinArea", "NeighborL2PinArea",
                             "NeighborL1BlkArea", "NeighborL2BlkArea", "NeighborMacroArea",
                             "NeighborMacroPinArea", "#NeighborCells", "#NeighborCellPins", "#NeighborMacros",
                             "#NeighborMacroPins", "#NeighborPassingNets"]

GRFeatures = ["#VerticalOverflow", "#VerticalRemain", "#VerticalTracks",
              "#HorizontalOverflow", "#HorizontalRemain", "#HorizontalTracks"]

GRNeighborFeatures = ["#NeighborVerticalOverflow", "#NeighborVerticalRemain", "#NeighborVerticalTracks",
                      "#NeighborHorizontalOverflow", "#NeighborHorizontalRemain", "#NeighborHorizontalTracks"]
selectedFeatures = set()
selectedFeatures.update(placementFeatures)
#selectedFeatures.update(placementNeighborFeatures)
#selectedFeatures.update(GRFeatures)
#selectedFeatures.update(GRNeighborFeatures)

AllDRVTypes = ["AdjacentCutSpacing", "SameLayerCutSpacing", "EndOfLine", "FloatingPatch", "MinArea", "MinWidth",
  "NonSuficientMetalOverlap", "CutShort", "MetalShort", "OutOfDieShort", "CornerSpacing", "ParallelRunLength"]

SelectedDRVTypes = ["CutShort", "MetalShort"]

label_name = "HasDetailedRoutingViolation"

# Selected Benchmarks

In [ ]:
# Benchmarks
# ispd18 = ["ispd18_test"+str(x) for x in range(1, 11)]
# ispd18.extend(["ispd18_test5_metal5", "ispd18_test8_metal5"]) #Include Hidden cases (benchmarks with less layers)

ispd19 = ["ispd19_test"+str(x) for x in range(1, 11)]
# ispd19.extend(["ispd19_test7_metal5", "ispd19_test8_metal5", "ispd19_test9_metal5"]) #Include Hidden cases (benchmarks with less layers)

circuits = ispd19
test_circuit = "ispd19_test10"
# test_circuit = 'swerv'
if test_circuit in circuits:
    circuits.remove(test_circuit)
circuits.remove("ispd19_test4")#low density benchmark
circuits.remove("ispd19_test5")#low density benchmark
circuits.remove("ispd19_test9")#Simillar from test10
circuits.extend(['aes_cipher_top', 'black_parrot', 'gcd', 'ibex', 'jpeg', 'dynamic_node', 'TinyRocket'])

# CSV Paths
csv_path = "/home/sheiny/workspace/Data/CSV/steiner_metal3/"
# csv_path = "./data/SecondIterationGCellCoords/"
# csv_path = "./data/FirstIterationFixedBin/"

# Data preprocessing

ATTENTION: If you want to deploy a model, it's critical that you preserve the preprocessing calculations.
The easiest way to implement them as layers, and attach them to your model before export.

In [ ]:
# The features will be rescaled so that they’ll have the properties of a standard normal distribution.
# mean (μ) = 0
# standard deviation (σ) = 1
def standardize(train_array, val_array, test_array=None):
    scaler = sklearn.preprocessing.StandardScaler()
    train_array = scaler.fit_transform(train_array)
    val_array = scaler.transform(val_array)
    if test_array is not None:
        test_array = scaler.transform(test_array)
        return train_array, val_array, test_array
    return train_array, val_array

def print_positive_ratio(train_labels):
    neg, pos = np.bincount(train_labels)
    total = neg + pos
    print('Examples:\n    Total: {}\n    Positive: {} ({:.2f}% of total)\n'.format(total, pos, 100 * pos / total))

# Claculate weight for classes
# Scaling by total/2 helps keep the loss to a similar magnitude.
# The sum of the weights of all examples stays the same.
def calculate_class_weights(df, train_labels):
    neg, pos = np.bincount(train_labels)
    total = neg + pos
    weight_for_0 = (1 / neg)*(total)/2.0 
    weight_for_1 = (1 / pos)*(total)/2.0
    class_weight = {0: weight_for_0, 1: weight_for_1}
    print('Weight for class 0: {:.2f}'.format(weight_for_0))
    print('Weight for class 1: {:.2f}'.format(weight_for_1))
    print_positive_ratio(train_labels)
    return class_weight, neg, pos

def oversample(train_array, train_labels):
    oversample = RandomOverSampler(sampling_strategy=0.5)
    train_array, train_labels = oversample.fit_resample(train_array, train_labels)
    print_positive_ratio(train_labels)
    return train_array, train_labels

def undersample(train_array, train_labels):
    undersample = RandomUnderSampler(sampling_strategy=0.5)
    train_array, train_labels = undersample.fit_resample(train_array, train_labels)
    print_positive_ratio(train_labels)
    return train_array, train_labels

# Some plot functions

In [ ]:
#matplotlib.rcParams['figure.figsize'] = (12, 10)
colors = matplotlib.pyplot.rcParams['axes.prop_cycle'].by_key()['color']

def plotDRVFrequency():
    benchmarks = ["ispd19_test"+str(x) for x in range(1, 11)]
    #Remove some non important DRVs
    drvs = AllDRVTypes.copy()
    drvs.remove("FloatingPatch")
    drvs.remove("MinWidth")
    drvs.remove("NonSuficientMetalOverlap")
    drvs.remove("CutShort")
    drvs.remove("OutOfDieShort")
    drv_dict = {drv:[] for drv in drvs}

    for circuit in benchmarks:
        df = pd.read_csv("./data/FirstIterationGCellCoords/"+circuit+".csv", dtype=np.float32)
        df2 = pd.read_csv("./data/SecondIterationGCellCoords/"+circuit+".csv", dtype=np.float32)
        for drv in drvs:
            drv_dict[drv].append((int(sum(df[drv])), int(sum(df2[drv]))))
    drv_df = pd.DataFrame(drv_dict)
    drv_df.index = benchmarks
    drv_df

def computeFScoreAndMCC(df):
    df['F-score'] = (2 * df['precision'] * df['recall'])/(df['precision'] + df['recall'])
    sqrt = np.sqrt((df['tp']+df['fp'])*(df['tp']+df['fn'])*(df['tn']+df['fp'])*(df['tn']+df['fn']))
    df['MCC'] = (df['tp'] * df['tn'] - df['fp'] * df['fn'])/sqrt

# plot the training loss and accuracy
def plot_df(history_df, metric, size=None):
    if size == None:
        size = history_df.shape[0]
    matplotlib.pyplot.style.use("ggplot")
    matplotlib.pyplot.figure()
    matplotlib.pyplot.plot(np.arange(0, size), history_df[metric][0:size], label=metric)
    matplotlib.pyplot.title("Training performace: "+metric)
    matplotlib.pyplot.xlabel("Epoch #")
    matplotlib.pyplot.ylabel(metric)
    matplotlib.pyplot.show()

def plot_cm(labels, predictions, title=None, output_path=None, p=0.5):
    cm = sklearn.metrics.confusion_matrix(labels, predictions > p)
    matplotlib.pyplot.figure(figsize=(5,5))
    seaborn.heatmap(cm, annot=True, fmt="d")
    if title == None:
        matplotlib.pyplot.title('Confusion matrix')
    else:
        matplotlib.pyplot.title(title)
    matplotlib.pyplot.ylabel('Actual label')
    matplotlib.pyplot.xlabel('Predicted label')
    if output_path != None:
        matplotlib.pyplot.savefig(output_path)
    else:
        matplotlib.pyplot.show()

# Load Training Data From ICCAD19

In [ ]:
test_df = pd.DataFrame()

dataframes = []
dataframes = [pd.read_csv(csv_path+circuit+".csv", dtype=np.float32) for circuit in circuits]
test_df = pd.read_csv(csv_path+test_circuit+".csv", dtype=np.float32)
    
#merge all DataFrames into a single one
df = pd.concat(dataframes, ignore_index=True)
#save some memory
dataframes.clear()

# Remove NodeIDs (debug info)
df = df.drop(columns=["NodeID"])
test_df_NodeID = test_df["NodeID"] #backup this for DRV draw
test_df = test_df.drop(columns=["NodeID"])

# Make sure to clear all DRV columns
df['HasDetailedRoutingViolation'] = False
test_df['HasDetailedRoutingViolation'] = False
# Apply filter for selected DRVs
for drv in SelectedDRVTypes:
    df['HasDetailedRoutingViolation'] = df['HasDetailedRoutingViolation'] | df[drv]
    test_df['HasDetailedRoutingViolation'] = test_df['HasDetailedRoutingViolation'] | test_df[drv]

# Drop all columns that are not in {selectedFeatures + label_name}
dontDropColumns = selectedFeatures | {label_name}
dropColumns = set(df.columns) - dontDropColumns
df = df.drop(columns=dropColumns)
test_df = test_df.drop(columns=dropColumns)

# Split 80/20 (train 80% test 20%)
train_df, val_df = sklearn.model_selection.train_test_split(df, test_size=0.2)

# Build np arrays of labels and features.
train_labels = np.array(train_df.pop('HasDetailedRoutingViolation'))
val_labels = np.array(val_df.pop('HasDetailedRoutingViolation'))
test_labels = np.array(test_df.pop('HasDetailedRoutingViolation'))
train_array = np.array(train_df)
val_array = np.array(val_df)
test_array = np.array(test_df)

# Apply the selected strategy to handle umbalanced data.
class_weight = None
if balance_strategy == BalanceStrategy.OVERSAMPLE:
    train_array, train_labels = oversample(train_array, train_labels)
elif balance_strategy == BalanceStrategy.UNDERSAMPLE:
    train_array, train_labels = undersample(train_array, train_labels)
elif balance_strategy == BalanceStrategy.WEIGHTS:
    class_weight = calculate_class_weights(df, train_labels)
    class_weight = class_weight[0]

# Scale
train_array, val_array, test_array = standardize(train_array, val_array, test_array)

# Load EhPredictor's dataset

# Load Model

In [ ]:
checkpoint_path = "./results/OverSampleSingleHiddenLayer50/cp.ckpt" #path to cp.ckpt
model = make_model()
model.load_weights(checkpoint_path)

# Train

In [ ]:
os.mkdir(model_name)
checkpoint_path = model_name+"/cp.ckpt"

# Create a callback that saves the model's weights at the end of each epoch
cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path, save_weights_only=True)
# Create a callback that saves model history at the end of each epoch
csv_logger = CSVLogger(model_name+"/model_history_log.csv", append=True)

#initialize learning model
neg, pos = np.bincount(train_labels)
initial_bias = np.log([pos/neg])
model = make_model(output_bias = initial_bias)

dataset = tf.data.Dataset.from_tensor_slices((train_array, train_labels))
train_dataset = dataset.shuffle(len(train_array)).batch(batch_size)

train_history = model.fit(train_dataset,
                          batch_size=batch_size,
                          validation_data=(val_array, val_labels),
                          class_weight=class_weight, #default class_weight = None
                          epochs=epochs,
                          callbacks=[cp_callback, csv_logger])

# Training performance

In [ ]:
history_df = pd.read_csv(model_name+"/model_history_log.csv") #Path to "model_history_log.csv"
computeFScoreAndMCC(history_df)
metrics_to_draw = ['loss', 'F-score', 'MCC', 'precision', 'recall']
max_epochs = 20 #Use None to draw the entire history
for metric in metrics_to_draw:
    plot_df(history_df, metric) #plot_df(history_df, metric, max_epochs)

# Test and check performance

In [ ]:
def calculate_test_metrics(model, results):
    m = {}
    for name, value in zip(model.metrics_names, results):
        m[name] = value
    if m['precision'] + m['recall'] != 0:
        f_score = (2 * m['precision'] * m['recall'])/(m['precision'] + m['recall'])
        m['F-score'] = f_score
    sqrt = math.sqrt((m['tp']+m['fp'])*(m['tp']+m['fn'])*(m['tn']+m['fp'])*(m['tn']+m['fn']))
    if sqrt != 0:
        mcc = (m['tp'] * m['tn'] - m['fp'] * m['fn'])/sqrt
        m['MCC'] = mcc
    return m

baseline_results = model.evaluate(test_array, test_labels, batch_size=batch_size, verbose=0)
metrics = calculate_test_metrics(model, baseline_results)
print(metrics)
test_predictions_baseline = model.predict(test_array, batch_size=batch_size)
plot_cm(test_labels, test_predictions_baseline, "Test Confusion Matrix")

# Save Predicted Node IDs to draw in OpenROAD

In [ ]:
# Save all predited node ids
# This can be used to plot the violating bins inside C++ OpenROAD
violations = test_predictions_baseline > 0.5
violating_ids = []
i = 0
for x in zip (violations, test_df_NodeID):
    if x[0]:
        violating_ids.append(int(x[1]))
with open(test_circuit+'violating_nodes.txt', 'w') as f:
    for item in violating_ids:
        f.write("%s\n" % item)